Ok so here's what I tried:
- put code for centered cov matrix into a function: 'center' (the function can also output a correlation matrix)
- the output is the cov matrix and also the rescaled Matrix
- the input for 'PCA' is changed to the outputs of 'center'
- instead of 'train_mean = None': 'center_test_values' (not happy with 2 very similar functions, will fix into 1 func 'center')

Advantages:
- seperate modules
- compare effects of corr mat vs cov mat (both work well so far!)

Disadvantages:
- less elegant (so far)

Thoughts and struggles and to do:
- correlation makes more sense here to me... ähnlicher Inhalt, mehr normalisiert. Sinnvoller auch wenn es wenig Unterschied für das Programm macht
- ~np.cov and np.corrcoef are kinda sus: I've written my own correlation function that works the same but np.cov is strange (see bottom of this doc!)~ fixed.
- own versions of np.cov and np.corrcoef are so SLOW 
- -> but they are necessary bc we want to apply centering and scaling from train values to the test values
- **Why X-mean???** Doesnt cov do that 4 u? The matrices are different! it should be redundant? but its not...
- test center function is still missing corr bit
- messy amount of variables in final code block

In [38]:
def center(X, scale = False):
    mean = np.mean(X, axis = 0)
    X_mean = X - mean
    cov_mat = np.cov(X_mean, rowvar = 0)
    #cov_mat = cov_columns(X_mean, X_mean)
    c_mat = cov_mat
    if scale:
        corr_mat = np.corrcoef(X_mean, rowvar=False)
        corr_mat[np.isnan(corr_mat)] = cov_mat[np.isnan(corr_mat)] # or replace with 0???
        c_mat = corr_mat
    return c_mat, X_mean

def center_test_values(X, Y, scale = False):
    mean = np.mean(Y, axis =0)
    X_mean = X - mean
    cov_mat = np.cov(X_mean, rowvar = 0)
    #cov_mat = cov_columns(X_mean, Y)
    c_mat = cov_mat
    if scale:
        x = len(Y[0])
        c = (1/(x))
        corr_mat = np.zeros((x, x))
        for i in range(0, x):
            for j in range (0, x):
                corr_mat[i,j] = c * np.dot(((X[:,i] - np.mean(Y[:,i]))/ np.std(Y[:,i])) , ((X[:,j] - np.mean(Y[:,j]))/ np.std(Y[:,j])))
        corr_mat[np.isnan(corr_mat)] = cov_mat[np.isnan(corr_mat)]
        c_mat = corr_mat
    return c_mat, X_mean

In [6]:
def PCA_func_mod(centered_x, X_mean, num_components, train_evs = None):
    # eigen values
    eigen_values, eigen_vectors = np.linalg.eigh(centered_x)
    # eigenvectors
    sorted_index = np.argsort(eigen_values)[::-1]
    sorted_eigenvalue = eigen_values[sorted_index]
    sorted_eigenvectors = eigen_vectors[:, sorted_index]
    # subset
    eigenvector_subset = sorted_eigenvectors[:, 0:num_components]
    # if we want to use the PCA that has been trained on the train set for the test set,
    # we want to use the eigenvector subset that has been calculated on the train set for classification
    # on the test set:
    if train_evs is not None:
        eigenvector_subset = train_evs
    X_reduced = np.dot(X_mean, eigenvector_subset)
    # we return X_reduced (the input reduced to num_components PCA),
    # the mean and the eigenvector subset (if we run the function for training, we need these for testing)
    return X_reduced, eigenvector_subset

## executable final project:

In [39]:
import numpy as np
import pickle
from functions.Load_data import load_the_pickle
from functions.PCA import PCA_func
from functions.KNN_predict import knn
#from functions.Standardize import center
#from functions.Standardize import center_test_values

# select number of principle components and k:
number_of_pcs = 45
k = 6

# loading data:
train_labels, train_values = load_the_pickle('data/train_points.p')
test_labels, test_values = load_the_pickle('data/test_points.p')


# PCA and standardization:
train_values_centered, train_mean = center(train_values)
train_values_pca, train_evs = PCA_func_mod(train_values_centered, train_mean, number_of_pcs)

test_values_centered, test_mean = center_test_values(test_values, train_values)
test_values_pca, _ = PCA_func_mod(test_values_centered,test_mean, number_of_pcs, train_evs=train_evs)

# kNN:
hit = 0
miss = 0

for i in range(10):
    sample = i
    predicted_value = knn(trainvalues_pca=train_values_pca, X=test_values_pca[sample, :], trainlabels=train_labels, k=k)
    labeled_value = test_labels[sample]
    if predicted_value == labeled_value:
        hit += 1
    else:
        miss += 1
print(hit, 'vs', miss)

KeyboardInterrupt: 

### Standardization experimentation below:

In [36]:
def corr_columns(mat):
    x = len(mat[0])
    c = (1/(x))
    corrmat = np.zeros((x, x))
    for i in range(0, x):
        for j in range (0, x):
            corrmat[i,j] = c * np.dot(((mat[:,i] - np.mean(mat[:,i]))/ np.std(mat[:,i])) , ((mat[:,j] - np.mean(mat[:,j]))/ np.std(mat[:,j])))
        
    return corrmat

In [37]:
def cov_columns(mat, mat2):
    x = len(mat2[0])
    c = (1/(x-1)) 
    covvmat = np.zeros((x, x))
    for i in range(0, x):
        for j in range (0, x):
            covvmat[i,j] = c * np.dot((mat[:,i] - np.mean(mat2[:,i])) , (mat[:,j] - np.mean(mat2[:,j])))
        
    return covvmat

In [27]:
# np.corrcoef uses "1/N" and np.cov uses "1/(N-1)"
mat_a = np.array([[1,2,3],[12,0,8], [6,0,4]])
print(corr_columns(mat_a))
print(np.corrcoef(mat_a, rowvar = False))
print(cov_columns(mat_a))
print(np.cov(mat_a, rowvar = 0))

[[ 1.         -0.83862787  0.96076892]
 [-0.83862787  1.         -0.65465367]
 [ 0.96076892 -0.65465367  1.        ]]
[[ 1.         -0.83862787  0.96076892]
 [-0.83862787  1.         -0.65465367]
 [ 0.96076892 -0.65465367  1.        ]]
[[30.33333333 -5.33333333 14.        ]
 [-5.33333333  1.33333333 -2.        ]
 [14.         -2.          7.        ]]
[[30.33333333 -5.33333333 14.        ]
 [-5.33333333  1.33333333 -2.        ]
 [14.         -2.          7.        ]]
